In [ ]:
import sys
sys.path.append('..')
from common.time_layers import *
from common.base_model import BaseModel

In [ ]:
class Encoder:
    def __init__(self,vocab_size,wordvec_size,hidden_size):
        #重みの初期化
        V,D,H=vocab_size,wordvec_size,hidden_size
        rn=np.random.randn

        embed_W=(rn(V,D)/100).astype('f')
        lstm_Wx=(rn(D,4*H)/np.sqrt(D)).astype('f')
        lstm_Wh=(rn(H,4*H)/np.sqrt(H)).astype('f')
        lstm_b=np.zeros(4*H).astype('f')

        #レイヤの生成
        self.embed=TimeEmbedding(embed_W)
        self.lstm=TimeLSTM(lstm_Wx,lstm_Wh,lstm_b,stateful=False)   #短い時系列データが複数ある問題のため、隠れ状態は維持しない

        #パラメータの格納
        self.params,self.grads=[],[]
        self.params=self.embed.params+self.lstm.params
        self.grads=self.embed.grads+self.lstm.grads
        self.hs=None

    def forward(self,xs):
        xs=self.embed.forward(xs)
        hs=self.lstm.forward(xs)
        self.hs=hs
        return hs[:,-1,:]   #最後の時刻の隠れ状態のみデコーダに伝える

    def backward(self,dh):
        dhs=np.zeros_like(self.hs)
        dhs[:,-1,:]=dh  #デコーダからの逆伝搬を最後の時刻の隠れ状態の勾配とする
        dout=self.lstm.backward(dhs)
        dout=self.embed.backward(dout)
        return dout

In [ ]:
class Decoder:
    def __init__(self,vocab_size,wordvec_size,hidden_size):
        V,D,H=vocab_size,wordvec_size,hidden_size
        rn=np.random.randn

        #重みの初期化
        embed_W=(rn(V,D)/100).astype('f')
        lstm_Wx=(rn(D,4*H)/np.sqrt(D)).astype('f')
        lstm_Wh=(rn(H,4*H)/np.sqrt(H)).astype('f')
        lstm_b=np.zeros(4*H).astype('f')
        affine_W=(rn(H,V)/np.sqrt(H)).astype('f')
        affine_b=np.zeros(V).astype('f')

        #レイヤの生成
        self.embed=TimeEmbedding(embed_W)
        self.lstm=TimeLSTM(lstm_Wx,lstm_Wh,lstm_b,stateful=True)    #エンコーダから受け取った隠れ状態を維持する
        self.affine=TimeAffine(affine_W,affine_b)

        #パラメータの格納
        self.params,self.grads=[],[]
        self.params=self.embed.params+self.lstm.params+self.affine.params
        self.grads=self.embed.grads+self.lstm.grads+self.affine.grads

    def forward(self,xs,h):
        self.lstm.set_state(h)
        out=self.embed.forward(xs)
        out=self.lstm.forward(out)
        score=self.affine.forward(out)
        return score

    def backward(self,dscore):
        dout=self.affine.backward(dscore)
        dout=self.lstm.backward(dout)
        dout=self.embed.backward(dout)
        dh=self.lstm.dh
        return dh

    def generate(self,h,start_id,sample_size):
        sampled=[]
        sample_id=start_id
        self.lstm.set_state(h)

        for _ in range(sample_size):
            x=np.array(sample_id).reshape((1,1))  #sampledを二次元の配列に変換してxに格納
            out=self.embed.forward(x)
            out=self.lstm.forward(out)
            score=self.affine.forward(out)

            sample_id=np.argmax(score.flatten())   #scoreを１次元のベクトルに変換して最大値のインデックスを取得
            sampled.append(int(sample_id))

        return sampled

In [ ]:
class Seq2seq(BaseModel):
    def __init__(self,vocab_size,wordvec_size,hidden_size):
        V,D,H=vocab_size,wordvec_size,hidden_size

        self.encoder=Encoder(V,D,H)
        self.decoder=Decoder(V,D,H)
        self.softmax=TimeSoftmaxWithLoss()

        self.params=self.encoder.params+self.decoder.params
        self.grads=self.encoder.grads+self.decoder.grads

    def forward(self,xs,ts):
        decoder_xs,decoder_ts=ts[:,:-1],ts[:,1:]    #tsが['_','6','2',' ',' ']のときdecoder_xsは['_','6','2',' ']、decoder_tsは['6','2',' ',' ']
        h=self.encoder.forward(xs)
        score=self.decoder.forward(decoder_xs,h)
        loss=self.softmax.forward(score,decoder_ts)
        return loss

    def backward(self,dout=1):
        dout=self.softmax.backward(dout)
        dh=self.decoder.backward(dout)
        dout=self.encoder.backward(dh)
        return dout

    def generate(self,xs,start_id,sample_size):
        h=self.encoder.forward(xs)
        sampled=self.decoder.generate(h,start_id,sample_size)
        return sampled